In [6]:
import dis

Piece of code to decompile

In [105]:
c = compile('''
x = 1
if x < 2:
    print(x)
''','','exec')

Decimal and Hex of the bytecode

In [103]:
' '.join([str(int(x)) for x in c.co_code])

'100 0 90 0 101 0 100 1 107 0 114 20 101 1 101 0 131 1 1 0 100 2 83 0'

In [104]:
' '.join([str(hex(x)) for x in c.co_code])

'0x64 0x0 0x5a 0x0 0x65 0x0 0x64 0x1 0x6b 0x0 0x72 0x14 0x65 0x1 0x65 0x0 0x83 0x1 0x1 0x0 0x64 0x2 0x53 0x0'

A couple helper functions

In [16]:
def group(lst, n):
    ''' Return an n-element touple generator from a list '''
    for i in range(0, len(lst), n):
        val = lst[i:i+n]
        if len(val) == n:
            yield tuple(val)

In [57]:
def group_with_idx(lst, n):
    ''' Return an n+n-element (element + indexes) touple generator from a list '''
    for i in range(0, len(lst), n):
        val = list(lst[i:i+n])
        if len(val) == n:
            yield tuple(val+[x for x in range(i,i+n)])

Tests of the above functions

In [58]:
[x for x in group([1,2,3,4,5,6],2)]

[(1, 2), (3, 4), (5, 6)]

In [106]:
[x for x in group_with_idx([1,2,3,4,5,6],2)]

[(1, 2, 0, 1), (3, 4, 2, 3), (5, 6, 4, 5)]

### Testing the Disassembly library

In [116]:
# Check what is in the disassembly library
print('\n'.join([x for x in dir(dis) if x[0] != '_']))

Bytecode
COMPILER_FLAG_NAMES
EXTENDED_ARG
FORMAT_VALUE
HAVE_ARGUMENT
Instruction
cmp_op
code_info
collections
dis
disassemble
disco
distb
findlabels
findlinestarts
get_instructions
hascompare
hasconst
hasfree
hasjabs
hasjrel
haslocal
hasname
hasnargs
io
opmap
opname
pretty_flags
show_code
stack_effect
sys
types


In [107]:
# Only Op Codes >= dis.HAVE_ARGUMENT have Op Arguments
dis.HAVE_ARGUMENT

90

In [115]:
# How many Op Codes are defined
len([x for x in dis.opname if "<" not in x])

119

In [117]:
# How many Op Codes are defined
len([x for x in dis.opname if "<" not in x])

119

In [118]:
# Longest Op Code name
max([len(x) for x in dis.opname if "<" not in x])

28

## Main piece of code

In [125]:
longest_opcode_name = max([len(dis.opname[op_code]) for op_code, _ in group(c.co_code,2)])

for op_code, op_arg, op_code_idx, op_arg_idx in group_with_idx(c.co_code,2):
    # Op Code
    op_code_idx_txt = f'[{op_code_idx:2d}]'
    op_code_txt     = f'{dis.opname[op_code]:{longest_opcode_name}s} ({op_code:3d})'
    
    # If the Op Code does not care about Op Argument, do not print anything else, otherwise print help
    op_arg_txt      =  ''
    op_arg_name     =  ''
    if op_code >= dis.HAVE_ARGUMENT:
        op_arg_txt  = f'{op_arg:3}'
        
        # Explanation of the arguments
        if op_code in dis.hasconst:
            op_arg_name = f'(const:{c.co_consts[op_arg]})'
        elif op_code in dis.hasname:
            op_arg_name = f'(name:{c.co_names[op_arg]})'
        elif op_code in dis.haslocal:
            op_arg_name = f'(local:{c.co_varnames[op_arg]})'
        elif op_code in dis.hasfree:
            op_arg_name = f'(free:{(c.co_cellvars+c.co_freevars)[op_arg]})'
        elif op_code in dis.hascompare:
            op_arg_name = f'(compare:{dis.cmp_op[op_arg]})'
        elif op_code in dis.hasjrel:
            op_arg_name = f'(rel jump: to {op_code_idx + 2 + op_arg})'
        elif op_code in dis.hasjabs:
            op_arg_name = f'(abs jump: to {op_arg})'
        elif op_code in dis.hasnargs:
            op_arg_name = f'(nargs:{op_arg})'
    
    # Print line
    print(op_code_idx_txt, op_code_txt,op_arg_txt,op_arg_name)

[ 0] LOAD_CONST        (100)   0 (const:1)
[ 2] STORE_NAME        ( 90)   0 (name:x)
[ 4] LOAD_NAME         (101)   0 (name:x)
[ 6] LOAD_CONST        (100)   1 (const:2)
[ 8] COMPARE_OP        (107)   0 (compare:<)
[10] POP_JUMP_IF_FALSE (114)  20 (abs jump: to 20)
[12] LOAD_NAME         (101)   1 (name:print)
[14] LOAD_NAME         (101)   0 (name:x)
[16] CALL_FUNCTION     (131)   1 
[18] POP_TOP           (  1)  
[20] LOAD_CONST        (100)   2 (const:None)
[22] RETURN_VALUE      ( 83)  


Inspired by https://towardsdatascience.com/understanding-python-bytecode-e7edaae8734d

---